# 作業工程計画

1. 受領データ②を整形
2.　受領データ③を整形
3.　受領データ④を整形
4.　1〜3をマージ

# ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

## 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

# エクセルデータの読み込み

## データ②

In [132]:
"""
1. 受領データ②を読み込む
2.　採点回ごとにデータフレームに入れる
3.　スタッフコードをキーにして一つのデータフレームとしてマージする
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/②採点者情報_採点に関する情報0510更新.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=list(range(0,24,1)))#headerを修正
df = df.rename(columns={'3始まり\nスタッフコード': 'スタッフコード3',
                        '採点回':'学年',
                        '割当単位':'割当',
                        'Unnamed: 3':'科目',
                        'Unnamed: 4':'登録試験点数',
                       '採否時\n登録時間':'採否時登録時間',
                       '1回目\n採点満点':'1回目採点満点',
                       '1回目\n採点\n得点':'1回目採点得点',
                       '混在\n枚数総数':'完全混在枚数総数',
                        "一致枚数":"完全一致枚数",
                        "一致率":"完全一致率",
                        "混在\n枚数総数.1":"範囲内混在枚数総数",
                        "一致枚数.1":"範囲内一致枚数",
                        "一致率.1":"範囲内一致率",
                       '枠更新率\n更新枠÷依頼枠':'枠更新率'})
df["年度"]= 21
df["採点回"]= "夏"
df_summer = df

In [101]:
df_summer.columns

Index(['スタッフコード', 'スタッフコード3', '氏名', '科目', '登録試験点数', '等級', '学年', '割当', '分野',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', '登録日',
       '依頼枚数', '枠更新率', '年度', '採点回'],
      dtype='object')

In [133]:
df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3,4] + list(range(24,43,1)))#headerを修正
df = df.rename(columns={'3始まり\nスタッフコード': 'スタッフコード3',
                        '等級.1':'等級',
                        '採点回.1':'学年',
                        '割当単位.1':'割当',
                        'Unnamed: 3':'科目',
                        'Unnamed: 4':'登録試験点数',                        
                        '分野.1':'分野',
                       '採否時\n登録時間.1':'採否時登録時間',
                        '1回目採点\n満点':'1回目採点満点',
                       '1回目\n採点満点.1':'1回目採点満点',
                       '1回目\n採点\n得点.1':'1回目採点得点',
                       '混在\n枚数総数.2':'完全混在枚数総数',
                        "一致枚数.2":"完全一致枚数",
                        "一致率.2":"完全一致率",
                        "混在\n枚数総数.3":"範囲内混在枚数総数",
                        "一致枚数.3":"範囲内一致枚数",
                        "一致率.3":"範囲内一致率",
                        '最終シール.1':'最終シール',
                        'ACTB.1':'ACTB',
                        'ACTB理由.1':'ACTB理由',
                        '登録日.1':'登録日',
                        '依頼枚数.1':'依頼枚数',
                       '枠更新率\n更新枠÷依頼枠.1':'枠更新率'})
df["年度"]= 21
df["採点回"]= "秋"
df_fall_f = df

In [103]:
df_fall_f.columns

Index(['スタッフコード', 'スタッフコード3', '氏名', '科目', '登録試験点数', '等級', '学年', '割当', '分野',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', '登録日',
       '依頼枚数', '枠更新率', '年度', '採点回'],
      dtype='object')

In [134]:
df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3,4] + list(range(43,62,1)))#headerを修正
df = df.rename(columns={'3始まり\nスタッフコード': 'スタッフコード3',
                        '等級.2':'等級',
                        '採点回.2':'学年',
                        '割当単位.2':'割当',
                        'Unnamed: 3':'科目',
                        'Unnamed: 4':'登録試験点数',                        
                        '分野.2':'分野',
                       '採否時\n登録時間.2':'採否時登録時間',
                       '1回目採点\n満点.1':'1回目採点満点',
                       # '1回目\n採点満点.2':'1回目採点満点',
                       '1回目\n採点\n得点.2':'1回目採点得点',
                       '混在\n枚数総数.4':'完全混在枚数総数',
                        "一致枚数.4":"完全一致枚数",
                        "一致率.4":"完全一致率",
                        "混在\n枚数総数.5":"範囲内混在枚数総数",
                        "一致枚数.5":"範囲内一致枚数",
                        "一致率.5":"範囲内一致率",
                        '最終シール.2':'最終シール',
                        'ACTB.2':'ACTB',
                        'ACTB理由.2':'ACTB理由',
                        '登録日.2':'登録日',
                        '依頼枚数.2':'依頼枚数',
                       '枠更新率\n更新枠÷依頼枠.2':'枠更新率'})
df["年度"]= 21
df["採点回"]= "秋"
df_fall_l = df

In [105]:
df_fall_l.columns

Index(['スタッフコード', 'スタッフコード3', '氏名', '科目', '登録試験点数', '等級', '学年', '割当', '分野',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', '登録日',
       '依頼枚数', '枠更新率', '年度', '採点回'],
      dtype='object')

In [135]:
df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3,4] + list(range(62,81,1)))#headerを修正
df = df.rename(columns={'3始まり\nスタッフコード': 'スタッフコード3',
                        '等級.3':'等級',
                        '採点回.3':'学年',
                        '割当単位.3':'割当',
                        'Unnamed: 3':'科目',
                        'Unnamed: 4':'登録試験点数',                        
                        '分野.3':'分野',
                       '採否時\n登録時間.3':'採否時登録時間',
                       '1回目\n採点\n満点':'1回目採点満点',
                       # '1回目\n採点満点.2':'1回目採点満点',
                       '1回目\n採点\n得点.3':'1回目採点得点',
                       '混在\n枚数総数.6':'完全混在枚数総数',
                        "一致枚数.6":"完全一致枚数",
                        "一致率.6":"完全一致率",
                        "混在\n枚数総数.7":"範囲内混在枚数総数",
                        "一致枚数.7":"範囲内一致枚数",
                        "一致率.7":"範囲内一致率",
                        '最終シール.3':'最終シール',
                        'ACTB.3':'ACTB',
                        'ACTB理由.3':'ACTB理由',
                        '登録日.3':'登録日',
                        '依頼枚数.3':'依頼枚数',
                       '枠更新率\n更新枠÷依頼枠.3':'枠更新率'})
df["年度"]= 21
df["採点回"]= "冬"
df_winter = df

In [109]:
df_winter.columns

Index(['スタッフコード', 'スタッフコード3', '氏名', '科目', '登録試験点数', '等級', '学年', '割当', '分野',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', '登録日',
       '依頼枚数', '枠更新率', '年度', '採点回'],
      dtype='object')

In [145]:
df =df_summer.append(df_fall_f)
df.append(df_fall_l)
df.append(df_winter)
df=df.drop('氏名', axis=1)
df=df.rename(columns={'科目':'科目コード'})
df_2 = df

In [146]:
df_2.columns

Index(['スタッフコード', 'スタッフコード3', '科目コード', '登録試験点数', '等級', '学年', '割当', '分野',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', '登録日',
       '依頼枚数', '枠更新率', '年度', '採点回'],
      dtype='object')

In [147]:
"""
Excelの書き出し 
"""

df = df_2

name_excel_output = "crlea2_dm_0510update_0602cleaned.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "crlea2_dm_0510update_0602cleaned.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

In [172]:
"""
Excelの書き出し 
"""

df = df_2

name_excel_output = "②採点者情報_採点に関する情報0510更新_加工.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "②採点者情報_採点に関する情報0510更新_加工.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## データ④

In [138]:
"""
1. 受領データ④を読み込む
2.　スタッフコードをキーにして一つのデータフレームとしてマージする
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/④採用者情報_個人に関わる詳細.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#headerを修正
df = df.rename(columns = {'学生/社会人':'身分'})
df_4 = df

In [120]:
df_4.head()

,スタッフコード,身分,性別,年齢,大学,学部,偏差値
0,1050015912,社会人,女性,58,東北大学医療技術短期大学部,NaN,短大
1,1050740209,社会人,女性,35,上智大学,文学部,69.6
2,1050742436,社会人,女性,53,岡山大学,教育学部,59.6
3,1050852853,社会人,女性,35,福岡女学院大学,人文学部,49
4,1051284876,社会人,女性,29,長崎大学,経済学部,56.6


In [148]:
df_24 = pandas_tool.merge(df_2,df_4,key = 'スタッフコード',how ='left' ,view=True)

df1のキー重複
10035
df2のキー重複
0
df1のオブザベーション:20070
df2のオブザベーション:14772
mergeのオブザベーション:20070


In [149]:
df_24.head()

,スタッフコード,スタッフコード3,科目コード,登録試験点数,等級,学年,割当,分野,採否時登録時間,1回目採点満点,...,依頼枚数,枠更新率,年度,採点回,身分,性別,年齢,大学,学部,偏差値
0,1051286194,3051286194,10,0,02,1年生,01,評論,108.75,NaN,...,5982,0.841525,21,夏,NaN,NaN,NaN,NaN,NaN,NaN
1,1051284876,3051284876,10,999,02,1年生,01,評論,97,NaN,...,8591,0.963683,21,夏,社会人,女性,29.0,長崎大学,経済学部,56.6
2,1060294576,3060294576,30,27,02,1年生,-,-,43.5,NaN,...,-,-,21,夏,NaN,NaN,NaN,NaN,NaN,NaN
3,1060309944,3060309944,10,26,02,1年生,01,評論,135,NaN,...,8711,1.008725,21,夏,社会人,女性,40.0,慶応義塾大学,文学部,71.4
4,1060311559,3060311559,10,25,02,1年生,01,評論,78,NaN,...,8342,1.013546,21,夏,NaN,NaN,NaN,NaN,NaN,NaN


## 分野の修正

In [151]:
#分野マスターは分野の表記揺れを統一したもの。 CRLEA様から修正版が送られてきたら、csvにして再度使用
"""
data1から分野を抜き出したcsvファイルを読み込む
"""

name_csv = "bunyamaster_v3.1.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/rawdata"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
bunya = pd.read_csv(filepath_or_buffer = path_file, sep=",")
bunya = bunya.rename(columns={'分野名':'分野'})
bunya = bunya.fillna({'解答言語': '日本語'})
bunya.head()

,科目コード,分野,分野名_修正v1,分野名_修正v2,ポイント採点,解答言語
0,10,評論,評論,評論,0,日本語
1,10,現代文・評論,評論,評論,0,日本語
2,10,現代文（評論）,評論,評論,0,日本語
3,10,現代文・評論（理由説明）,評論,評論,0,日本語
4,10,ポ評論,評論,評論,1,日本語


In [152]:
"""
2つのテーブルを結合させる
df_1:
bunya:分野の表記揺れをなおした列（分野名（修正））がある
"""
df1 = df_24
df2 = bunya
df_24b = pandas_tool.merge(df1,df2, key = ["分野","科目コード"],how = 'left')
# df_data2b.drop(['氏名', '登録試験点数'], axis=1)
df_24b.head()

df1のキー重複
20001
df2のキー重複
0
df1のオブザベーション:20070
df2のオブザベーション:134
mergeのオブザベーション:20070


,スタッフコード,スタッフコード3,科目コード,登録試験点数,等級,学年,割当,分野,採否時登録時間,1回目採点満点,...,身分,性別,年齢,大学,学部,偏差値,分野名_修正v1,分野名_修正v2,ポイント採点,解答言語
0,1051286194,3051286194,10,0,02,1年生,01,評論,108.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,評論,評論,0.0,日本語
1,1051284876,3051284876,10,999,02,1年生,01,評論,97,NaN,...,社会人,女性,29.0,長崎大学,経済学部,56.6,評論,評論,0.0,日本語
2,1060294576,3060294576,30,27,02,1年生,-,-,43.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1060309944,3060309944,10,26,02,1年生,01,評論,135,NaN,...,社会人,女性,40.0,慶応義塾大学,文学部,71.4,評論,評論,0.0,日本語
4,1060311559,3060311559,10,25,02,1年生,01,評論,78,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,評論,評論,0.0,日本語


## データ③

In [169]:
"""
1. 受領データ③を読み込む
2.　採点者コード（スタッフコード）、採点回、学年をキーにしてdf_2のデータフレームとマージする
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/③【21夏-冬】採点者別ペース_0414更新.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list= input_file.sheet_names
sn_list = sn_list[0:len(sn_list)-2]
#配列ループ
df = [] #空のリスト
for i in range(0,len(sn_list)): 

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], index_col=None)
    #ret
    ret["採点回"]= sn_list[i][0]#sn_listのiつ目の要素から1番目の文字
    ret["学年"]=int(sn_list[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

map_dictionary ={1 : "1年生", 2 :"2年生", 3 : "3年生"} 
dfs['学年'] = dfs['学年'].map(map_dictionary) 
dfs = dfs.rename(columns = {'合計 / 合計 / バツウチ済枚数':'バツウチ済枚数',
                         '合計 / 合計 / 採点済枚数':'採点済枚数',
                         '合計 / 合計 / 採点所要時間':'採点所要時間',
                         'ペース':'最終ペース',
                           '採点者コード':'スタッフコード3'})
dfs=dfs[dfs['科目コード'].isin([10, 20,30])]#科目コード10、20、30を抽出
df_3 = dfs

In [170]:
df_3['学年'].unique()

array(['1年生', '2年生', '3年生'], dtype=object)

In [171]:
"""
2つのテーブルを結合させる
df_1:
bunya:分野の表記揺れをなおした列（分野名（修正））がある
"""
df1 = df_24b
df2 = df_3
df_24b3 = pandas_tool.merge(df1,df2, key = ["スタッフコード3","採点回","学年","科目コード"],how = 'left')
# df_data2b.drop(['氏名', '登録試験点数'], axis=1)
df_24b3.head()

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:20070
df2のオブザベーション:12376
mergeのオブザベーション:20070


,スタッフコード,スタッフコード3,科目コード,登録試験点数,等級,学年,割当_x,分野,採否時登録時間,1回目採点満点,...,分野名_修正v2,ポイント採点,解答言語,製品コード,割当_y,採点所要時間,採点済枚数,バツウチ済枚数,実枚数,最終ペース
0,1051286194,3051286194,10,0,02,1年生,01,評論,108.75,NaN,...,評論,0.0,日本語,111103.0,1.0,10938.0,5662.0,651.0,5011.0,1649.259462
1,1051284876,3051284876,10,999,02,1年生,01,評論,97,NaN,...,評論,0.0,日本語,111103.0,1.0,177905.0,9666.0,1076.0,8590.0,173.823108
2,1060294576,3060294576,30,27,02,1年生,-,-,43.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1060309944,3060309944,10,26,02,1年生,01,評論,135,NaN,...,評論,0.0,日本語,111103.0,1.0,71272.0,10081.0,1371.0,8710.0,439.948367
4,1060311559,3060311559,10,25,02,1年生,01,評論,78,NaN,...,評論,0.0,日本語,111103.0,1.0,86555.0,9535.0,1193.0,8342.0,346.960892


In [174]:
"""
Excelの書き出し 
"""

df = df_24b3

name_excel_output = "crlea_saiten_dm_0510update_0602cleaned.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "crlea_saiten_dm_0510update_0602cleaned.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)